# Практика 7

## Библиотеки

In [34]:
import pandas as pd


### 1. Найти данные для задачи классификации

In [35]:
mushrooms = pd.read_csv('C:\\Users\\grman\\Documents\\BigPapa\\Greg\\BigData\\DataSet\\Pr7\\mushrooms.csv')

In [36]:
mushrooms.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [37]:
mushrooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [38]:
mushrooms.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

### 2. Реализовать баггинг

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


columns = ["class", "cap-shape", "cap-surface", "cap-color", "bruises", "odor", "gill-attachment", "gill-spacing",
           "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring",
           "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color",
           "ring-number", "ring-type", "spore-print-color", "population", "habitat"]

mapping = {
    'p': 0, # posinous
    'e': 1, # edible
}
def replace_tf(mush):
    return mapping.get(mush, mush)

# Преобразование категориальных признаков в числовые
mushroom_data_encoded = pd.get_dummies(mushrooms)

mushroom_data_encoded['class'] = mushrooms['class'].apply(replace_tf)
mushroom_data_encoded['class'] = mushroom_data_encoded['class'].apply(int)
mushroom_data_encoded.drop(columns=['class_e','class_p'],axis=0)


X = mushroom_data_encoded.drop(columns=['class'], axis=1)
y = mushroom_data_encoded['class']

In [40]:

# training set and testing set of data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение базового классификатора (дерево решений)
base_classifier = DecisionTreeClassifier()

# Определение количества моделей в ансамбле
n_classifiers = 15

# Создание и обучение баггинг классификатора
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=n_classifiers, random_state=42)
bagging_classifier.fit(X_train, y_train)

# Получение прогнозов на тестовом наборе данных
y_pred = bagging_classifier.predict(X_test)

# Оценка точности модели
accuracy1 = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy1}')


Accuracy: 1.0


In [41]:

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

base_classifier = DecisionTreeClassifier(max_depth=1)  # Ограничиваем глубину дерева для AdaBoost

# Определение количества моделей в ансамбле
n_classifiers = 50

# Создание и обучение бустинг классификатора (AdaBoost)
boosting_classifier = AdaBoostClassifier(base_classifier, n_estimators=n_classifiers, random_state=42)
boosting_classifier.fit(X_train, y_train)

# Получение прогнозов на тестовом наборе данных
y_pred = boosting_classifier.predict(X_test)

# Оценка точности модели
accuracy2 = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy2}')


Accuracy: 1.0


In [42]:
from sklearn.metrics import f1_score, confusion_matrix

precision = [0,0]
f1 = [0,0]

def error_cal(y,i):
    cm = confusion_matrix(y_test, y)
    #матрицы ошибок 
    TP = cm[1, 1]
    FP = cm[0, 1]
    FN = cm[1, 0]
    
    precision[i] = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1[i] = 2 * (precision[i] * recall) / (precision[i] + recall)

error_cal(bagging_classifier.predict(X_test),0)
error_cal(boosting_classifier.predict(X_test),1)

In [43]:
mushrooms.count()

class                       8124
cap-shape                   8124
cap-surface                 8124
cap-color                   8124
bruises                     8124
odor                        8124
gill-attachment             8124
gill-spacing                8124
gill-size                   8124
gill-color                  8124
stalk-shape                 8124
stalk-root                  8124
stalk-surface-above-ring    8124
stalk-surface-below-ring    8124
stalk-color-above-ring      8124
stalk-color-below-ring      8124
veil-type                   8124
veil-color                  8124
ring-number                 8124
ring-type                   8124
spore-print-color           8124
population                  8124
habitat                     8124
dtype: int64

In [44]:
results = pd.DataFrame({
    'Algorithm': ['Bagging', 'Boosting'],
    'Accuracy': [accuracy1, accuracy2],
    'Precision': [precision[0],precision[1]],
    'F1-Score':[f1[0],f1[1]]
})

print(results)

  Algorithm  Accuracy  Precision  F1-Score
0   Bagging       1.0        1.0       1.0
1  Boosting       1.0        1.0       1.0


Если точность обеих моделей (бэггинга и бустинга) составляет 1.00 (или 100%), это может означать, что обе модели идеально справились с задачей классификации на выбранных данных. В данном случае:

Бэггинг (Random Forest):

Бэггинг, основанный на случайном лесе, показал высокую точность, равную 1.00.
Это может свидетельствовать о стабильности и надежности модели на предоставленных данных.
Бустинг (Gradient Boosting):

Градиентный бустинг также достиг идеальной точности в 1.00.
Это указывает на успешное последовательное улучшение модели за счет коррекции ошибок предыдущих моделей.
В таком случае, выбор между бэггингом и бустингом может зависеть от других факторов, таких как время обучения, сложность моделей и их интерпретируемость. Возможно, несмотря на идеальные результаты, один из методов может быть более выгодным с точки зрения вычислительной эффективности или простоты модели.